# Imports

In [37]:
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta
import functools
import time
from tqdm import tqdm
import numpy as np
from dateutil.relativedelta import relativedelta
from time import sleep


In [38]:
import sys
ROOT = '../'
sys.path.append(ROOT)  # Add the root folder to the sys.path

# Import the modules
from config import *

# Reload the configuration
from importlib import reload
reload(sys.modules['config'])

# Import the reloaded modules
from config import *

# Blockchain

In [ ]:
timespan = "10years" # Default: TIMESPAN | Max: "10years" 
end_date = dt.strptime(END_DATE, "%Y-%m-%d")
start_date = dt.strptime("2011-02-28", "%Y-%m-%d") # Default: dt.strptime(START_DATE, "%Y-%m-%d")

# Add one year to the end date to make sure we get all the data
limit = (end_date - start_date).days + relativedelta(years=1).days # Default: (end_date - start_date).days + relativedelta(years=1).days

print("Start date: ", start_date)
print("End date: ", end_date)
print("Timespan: ", timespan)
print("Limit: ", limit)

In [65]:
# Metrics considered
metrics = [
          # Block Details
          "blocks-size",
          "avg-block-size",
          "n-transactions-total",
          "n-transactions-per-block",

          # Mining Information
          "hash-rate",
          "difficulty",
          "miners-revenue",
          "transaction-fees-usd",

          # Network Activity
          "n-unique-addresses",
          "n-transactions",
          "estimated-transaction-volume-usd"
]

In [66]:
def blockchain_data_crawler(timespan, metric, start):
    # API info
    url = f'https://api.blockchain.info/charts/{metric}?timespan={timespan}&start={start}&format=csv'

    # Obtain data
    data = pd.read_csv(url, names=['timestamp', metric])

    # Transform "timestamp" to datetime type
    data['timestamp'] = pd.to_datetime(data["timestamp"])

    return data

In [ ]:
# Merge the data
merge = functools.partial(pd.merge, on='timestamp')

# Gain blockchain data from Blockchain.com API
blockchain_data_raw = functools.reduce(merge, [blockchain_data_crawler(timespan, metric, start_date.strftime('%Y-%m-%d')) for metric in metrics])
blockchain_data_raw

In [68]:
# If the data is not complete, we need to get the rest of the data (until the end date)
if blockchain_data_raw['timestamp'].iloc[-1] < end_date:
    # Get the last timestamp
    last_timestamp = blockchain_data_raw['timestamp'].iloc[-1]

    # Get the rest of the data from the last timestamp until the end date
    blockchain_data_rest = functools.reduce(merge, [blockchain_data_crawler(timespan, metric, last_timestamp.strftime('%Y-%m-%d')) for metric in metrics])

    # Concatenate the data
    blockchain_data = pd.concat([blockchain_data_raw, blockchain_data_rest])
else:
    blockchain_data = blockchain_data_raw


In [ ]:
blockchain_data

In [ ]:
# Plot the dates
import matplotlib.pyplot as plt

plt.plot(blockchain_data['timestamp'], blockchain_data['n-transactions'])
plt.show()

In [77]:
# Select only the data from 2016 onwards
blockchain_data = blockchain_data[blockchain_data['timestamp'] >= '2016-01-01'].reset_index(drop=True)


In [ ]:
# Plot the dates
plt.plot(blockchain_data['timestamp'], blockchain_data['n-transactions'])
plt.show()

In [79]:
blockchain_data_1_copy = blockchain_data.copy()

# Convert the timestamp column to datetime format
blockchain_data_1_copy['timestamp'] = pd.to_datetime(blockchain_data_1_copy['timestamp'])

In [ ]:
# Check duplicated rows on timestamp column
duplicated_rows = blockchain_data_1_copy[blockchain_data_1_copy.duplicated(subset=['timestamp'], keep=False)]
print("Number of duplicated rows: ", duplicated_rows.shape[0])

# Drop duplicated rows
blockchain_data_1_copy = blockchain_data_1_copy.drop_duplicates(subset=['timestamp'], keep='first')
blockchain_data_1_copy

In [ ]:
# Check missing values
missing_values = blockchain_data_1_copy.isnull().sum()
print("Missing values: ", missing_values)

In [ ]:
# Retrieving market capitalization and total circulating data
metrics = [
          # Currency Statistics
          "total-bitcoins",
          "market-cap",
  ]

blockchain_data_raw = functools.reduce(merge, [blockchain_data_crawler(timespan, metric, start_date.strftime('%Y-%m-%d')) for metric in metrics])
blockchain_data_raw

In [83]:
# If the data is not complete, we need to get the rest of the data (until the end date)
if blockchain_data_raw['timestamp'].iloc[-1] < end_date:
    # Get the last timestamp
    last_timestamp = blockchain_data_raw['timestamp'].iloc[-1]

    # Get the rest of the data
    blockchain_data_rest = functools.reduce(merge, [blockchain_data_crawler(timespan, metric, last_timestamp.strftime('%Y-%m-%d')) for metric in metrics])

    # Concatenate the data
    blockchain_data = pd.concat([blockchain_data_raw, blockchain_data_rest])
else:
    blockchain_data = blockchain_data_raw

In [ ]:
blockchain_data

In [ ]:
# Plot the dates
plt.plot(blockchain_data['timestamp'], blockchain_data['market-cap'])
plt.show()

In [ ]:
blockchain_data_2_copy = blockchain_data.copy()

# Extract the date part (year-month-day) and convert it to datetime format
blockchain_data_2_copy['timestamp'] = blockchain_data_2_copy['timestamp'].dt.date
blockchain_data_2_copy['timestamp'] = pd.to_datetime(blockchain_data_2_copy['timestamp'])
blockchain_data_2_copy

In [ ]:
# Check duplicated rows on timestamp column
duplicated_rows = blockchain_data_2_copy[blockchain_data_2_copy.duplicated(subset=['timestamp'], keep=False)]
print("Number of duplicated rows: ", duplicated_rows.shape[0])

# Drop duplicated rows
blockchain_data_2_copy = blockchain_data_2_copy.drop_duplicates(subset=['timestamp'], keep='first')
blockchain_data_2_copy

In [ ]:
# Check missing values
missing_values = blockchain_data_2_copy.isnull().sum()
print("Missing values: ", missing_values)

In [89]:
# Merge all data
blockchain_data = pd.merge(blockchain_data_1_copy, blockchain_data_2_copy, how="left", on='timestamp')

In [ ]:
blockchain_data

In [ ]:
# Check missing values
blockchain_data.isnull().sum()

In [93]:
# Generate the daily and hourly datasets
start_date = blockchain_data['timestamp'].iloc[0] # Default: START_DATE
blockchain_daily = pd.date_range(start=start_date, end=END_DATE, freq='D').to_frame(index=False, name='timestamp')
blockchain_hourly = pd.date_range(start=start_date, end=END_DATE, freq='h').to_frame(index=False, name='timestamp')

In [ ]:
blockchain_daily

In [ ]:
# blockchain_hourly

In [ ]:
# Merge all data with the daily and hourly datasets
blockchain_data_daily = pd.merge(blockchain_daily, blockchain_data, how="left", on='timestamp')
# blockchain_data_hourly = pd.merge(blockchain_hourly, blockchain_data, how="left", on='timestamp')

# Fill missing values with the previous value
blockchain_data_daily = blockchain_data_daily.fillna(method='ffill')
# blockchain_data_hourly = blockchain_data_hourly.fillna(method='ffill')

In [ ]:
blockchain_data_daily

In [ ]:
# blockchain_data_hourly

In [99]:
# Save the data to a CSV file
output_file = os.path.join(ROOT, PRICE_BLOCKCHAIN_DATASET_PATH, "blockchain_daily.csv")
blockchain_data_daily.to_csv(output_file, index=False)

In [101]:
# # Plot the data
# import matplotlib.pyplot as plt

# fig, ax = plt.subplots(1, 1, figsize=(15, 5))
# ax.plot(blockchain_data_upsampled['timestamp'], blockchain_data_upsampled['market-cap'], label='Market Cap')
# ax.set_title('Market Capitalization')
# ax.set_xlabel('Date')
# ax.set_ylabel('Market Cap')
# ax.legend()

# plt.show()

In [ ]:
# # Save the data to a CSV file
# output_file = os.path.join(ROOT, PRICE_BLOCKCHAIN_DATASET_PATH, "blockchain_hourly.csv")
# blockchain_data_hourly.to_csv(output_file, index=False)

# OHLCV

In [107]:
import datetime
import calendar
import requests
import pandas as pd
import os.path
import time

In [108]:
# Save the data to a CSV file
output_file = os.path.join(ROOT, PRICE_BLOCKCHAIN_DATASET_PATH, 'ohlcv_noheader.csv')

if (os.path.isfile(output_file)): #if the file already exists start from the latest date
    starttime = datetime.datetime.fromtimestamp(int(str(int(pd.read_csv(output_file, header=None).iloc[-1][0]))[:-3])) # read the last timestamp for csv file. Bitstamp takes and returs date date with 3 extra zeros. So that
else:
    start_date = "2016-01-01" # Default: START_DATE 
    starttime = datetime.datetime.strptime(start_date, '%Y-%m-%d') #Start collecting from start_date

start_unixtime = calendar.timegm(starttime.utctimetuple())

latest_time = int(time.time() - 60 * 60 * 24) #The real ending time. Collect data from starttime to current time - 24 hours

track_time = time.time() #because bitstamp only allows 10 requests per minute. Take rest if we are faster than that
count = 0

In [ ]:
while (start_unixtime < latest_time):
    end_unixtime = start_unixtime + 60*60*24*30 # 30 days at a time
    
    if (end_unixtime > latest_time):
        end_unixtime = latest_time # If the time is in future.

    url = 'https://api.bitfinex.com/v2/candles/trade:1h:tBTCUSD/hist?start={}&end={}&limit=1000'.format(str(start_unixtime) + "000", str(end_unixtime) + "000") # 1 hour can be changed to any timeframe
    response = requests.get(url)
    data = response.json()

    ohlcv_data_raw = pd.DataFrame(data).set_index(0).sort_index() # Set the date column as index and sort all data

    ohlcv_data_raw.to_csv(output_file, header=None,mode='a') # Append the data
    
    print('Saved till {}'.format(datetime.datetime.fromtimestamp(int(end_unixtime)).strftime('%Y-%m-%d %H:%M:%S')))
    
    start_unixtime = end_unixtime + 60 * 60 # To prevent duplicates
    count = count + 1
    
    if (count == 10): # If 10 requests are made
        count = 0 # Reset it
        
        diff = time.time() - track_time
        
        if (diff <= 60):
            print('Sleeping for {} seconds'.format(str(60 - diff)))
            time.sleep(60 - diff) #sleep
            
        
        track_time = time.time()
    # Bitstamp limits to 10 requests per minute

In [ ]:
ohlcv_data_raw

In [ ]:
# Add the header
ohlcv_data_raw = pd.read_csv(output_file, header=None, index_col=None)

ohlcv_data = ohlcv_data_raw.copy()
ohlcv_data.columns = ['timestamp', 'open', 'close', 'high', 'low', 'volume']
ohlcv_data.set_index('timestamp') 

# Convert unix into %Y-%m-%d %H:%M:%S format
ohlcv_data['timestamp'] = pd.to_datetime(ohlcv_data['timestamp'], unit='ms')
ohlcv_data

In [ ]:
# Check missing values
missing_values = ohlcv_data.isnull().sum()
print("Missing values: ", missing_values)

In [ ]:
# Check duplicated rows on timestamp column
duplicated_rows = ohlcv_data[ohlcv_data.duplicated(subset=['timestamp'], keep=False)]
print("Number of duplicated rows: ", duplicated_rows.shape[0])

# Drop duplicated rows
ohlcv_data = ohlcv_data.drop_duplicates(subset=['timestamp'], keep='first')
ohlcv_data

In [ ]:
# Crop the data from START_DATE to END_DATE
# ohlcv_data = ohlcv_data[(ohlcv_data['timestamp'] >= START_DATE) & (ohlcv_data['timestamp'] <= END_DATE)]
# ohlcv_data

In [ ]:
# Set the timestamp as the index
ohlcv_data = ohlcv_data.set_index('timestamp', drop=False)

ohlcv_data_downsampled = ohlcv_data.resample('1d').ffill()
ohlcv_data_downsampled

In [115]:
# Save the data to a CSV file
output_file = os.path.join(ROOT, PRICE_BLOCKCHAIN_DATASET_PATH, 'ohlcv_daily.csv')
ohlcv_data_downsampled.to_csv(output_file, index=False)

In [ ]:
# ohlcv_data_upsampled = ohlcv_data.resample('1h').ffill()
# ohlcv_data_upsampled

In [ ]:
# # Plot the data
# import matplotlib.pyplot as plt

# plt.figure(figsize=(14, 7))
# plt.plot(ohlcv_data['timestamp'], ohlcv_data['close'])
# plt.title("BTC/USDT Close Price")
# plt.xlabel("Date")
# plt.ylabel("Price (USDT)")
# plt.grid()
# plt.show()

In [ ]:
# # Save the data to a CSV file
# output_file = os.path.join(ROOT, PRICE_BLOCKCHAIN_DATASET_PATH, 'ohlcv_hourly.csv')
# ohlcv_data_upsampled.to_csv(output_file, index=False)

# Merge data

In [120]:
# Load data
blockchain_daily = pd.read_csv(os.path.join(ROOT, PRICE_BLOCKCHAIN_DATASET_PATH, 'blockchain_daily.csv'))
ohlcv_daily = pd.read_csv(os.path.join(ROOT, PRICE_BLOCKCHAIN_DATASET_PATH, 'ohlcv_daily.csv'))

# ohlcv_hourly = pd.read_csv(os.path.join(ROOT, PRICE_BLOCKCHAIN_DATASET_PATH, 'ohlcv_hourly.csv'))
# blockchain_hourly = pd.read_csv(os.path.join(ROOT, PRICE_BLOCKCHAIN_DATASET_PATH, 'blockchain_hourly.csv'))

In [ ]:
blockchain_daily

In [ ]:
ohlcv_daily

In [132]:
# Make sure that the timestamp column format is the same in both datasets
blockchain_daily['timestamp'] = pd.to_datetime(blockchain_daily['timestamp'])
ohlcv_daily['timestamp'] = pd.to_datetime(ohlcv_daily['timestamp'])

In [133]:
# Merge ohlcv and blockchain data
merged_daily = pd.merge(ohlcv_daily, blockchain_daily, how="inner", on='timestamp')
# merged_hourly = pd.merge(ohlcv_hourly, blockchain_hourly, how="inner", on='timestamp')

In [ ]:
merged_daily

In [ ]:
# merged_hourly

In [ ]:
# Check duplicates
merged_daily[merged_daily.duplicated(subset=['timestamp'], keep=False)]

In [ ]:
# Check missing values
merged_daily.isnull().sum()

In [137]:
# Save the data to a CSV file
output_file = os.path.join(ROOT, PRICE_BLOCKCHAIN_DATASET_PATH, "price_blockchain_daily.csv")
merged_daily.to_csv(output_file, index=False)

# output_file = os.path.join(ROOT, PRICE_BLOCKCHAIN_DATASET_PATH, "price_blockchain_hourly.csv")
# merged_hourly.to_csv(output_file, index=False)